In [313]:
#Essentials
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv 
import datetime as dt

#For scraping Reddit
import praw
from psaw import PushshiftAPI

#For EDA
import re
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.cluster import KMeans
import sklearn.cluster
import sklearn.decomposition
import matplotlib.colors
from textblob import TextBlob
from gensim.models import Word2Vec
import nltk
import nltk.corpus
from nltk.corpus import nps_chat
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import NaiveBayesClassifier
from nltk import TweetTokenizer
import textstat
import gensim.corpora as corpora
from pprint import pprint
import gensim
from gensim.utils import simple_preprocess
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis
import os
import emot
import demoji

In [78]:
nltk.download('punkt')
nltk.download('nps_chat')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Julia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package nps_chat to
[nltk_data]     C:\Users\Julia\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\nps_chat.zip.


True

# Mental Heatlh

In [2]:
#Thanks to dmarx for the clear code on pulling from PSAW! PSAW docs are surprisingly unclear.
#https://github.com/dmarx/psaw
api = PushshiftAPI()

#Start date we're beginning to pull posts from
start_epoch=int(dt.datetime(2010, 1, 1).timestamp())


#Pulling submissions from mentalhealth (both the title and body of a post)

gen_subMH = list(api.search_submissions(after=start_epoch,
                            subreddit='mentalhealth',
                            filter=['title', 'id','score','link_flair_text','selftext'],
                            limit=1000000))


#Converting to dataframe
MHsub = pd.DataFrame([thing.d_ for thing in gen_subMH])


C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


In [3]:
MHsub.to_csv('../Data/MHsub.csv',index=False)

In [4]:
#Pulling comments from mental health

gen_comMH = api.search_comments(after=start_epoch, 
                            subreddit='mentalhealth',
                            filter=['body', 'link_id', 'id', 'score','submission'],
                            #link_id = new_idsMH,
                            limit=35000000)

MHcom = pd.DataFrame([thing.d_ for thing in gen_comMH])

C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


In [5]:
MHcom.to_csv('../Data/MHcom.csv',index=False)

# Mental Illness subreddit

In [6]:
gen_subMI = list(api.search_submissions(after=start_epoch,
                            subreddit='mentalillness',
                            filter=['title', 'id','score','link_flair_text','selftext'],
                            limit=1000000))


#Converting to dataframe
MIsub = pd.DataFrame([thing.d_ for thing in gen_subMI])

C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


In [7]:
MIsub.to_csv('../Data/MIsub.csv',index=False)

In [8]:
gen_comMI = api.search_comments(after=start_epoch, 
                            subreddit='mentalillness',
                            filter=['body', 'link_id', 'id', 'score','submission'],
                            #link_id = new_MIids,
                            limit=35000000)

MIcom = pd.DataFrame([thing.d_ for thing in gen_comMI])

C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


In [9]:
MIcom.to_csv('../Data/MIcom.csv',index=False)

# Loading the data

In [223]:
MHsub = pd.read_csv('../Data/MHsub.csv')
MHcom = pd.read_csv('../Data/MHcom.csv')
MIsub = pd.read_csv('../Data/MIsub.csv')
MIcom = pd.read_csv('../Data/MIcom.csv')

# Cleaning the data

# We can merge submissions from mentalhealth and mentalillness subreddits

In [224]:
all_sub = pd.concat([MHsub,MIsub])
all_sub.reset_index(inplace=True,drop=True)

In [225]:
all_sub

,created_utc,id,score,selftext,title,created,link_flair_text
0,1663534517,xhrtbf,1,I don’t entirely want to reach out to people h...,How do I stop villainizing myself.,1.663549e+09,NaN
1,1663534244,xhrpbh,1,"I suffer from anxiety attacks regularly, usual...",is this a panic attack or something else?,1.663549e+09,NaN
2,1663533992,xhrlh6,1,I've been having these thoughts lately like\n\...,do you ever feel like not being alive anymore,1.663548e+09,NaN
3,1663533274,xhrats,1,"For some miraculous reason, I got a girlfriend...",Being normal,1.663548e+09,Opinion / Thoughts
4,1663533255,xhrajv,1,I've been going through a hard time lately and...,why do depressed people shave their heads,1.663548e+09,NaN
...,...,...,...,...,...,...,...
415285,1353891546,13sbe9,1,NaN,I look again to make sure so one is near me an...,1.353906e+09,NaN
415286,1348724331,10juc6,1,NaN,"NAMI | Peer-to-Peer is a unique, experiential ...",1.348739e+09,NaN
415287,1344639905,y0sqy,0,AMA,I am rooster illusion,1.344654e+09,NaN
415288,1321506737,mfckr,1,so i have a lot of random impluses. crazy shit...,crazy motherfucker,1.321521e+09,NaN


In [226]:
all_sub['created'] = pd.to_datetime(all_sub['created'], unit='s')

In [227]:
all_sub = all_sub.drop(columns=['created_utc','score','link_flair_text'])

In [228]:
all_sub = all_sub.rename(columns={'selftext': 'post_body'})

In [234]:
#making the title & post_body lowercase for future processing.
all_sub['post_body'] = all_sub['post_body'].astype(str)
all_sub['title'] = all_sub['title'].astype(str)
all_sub['title'] = all_sub['title'].apply(lambda x: x.lower())
all_sub['post_body'] = all_sub['post_body'].apply(lambda x: x.lower())

# Merging two dataframes with comments

In [236]:
all_com = pd.concat([MHcom,MIcom])

In [237]:
all_com

,body,created_utc,id,link_id,score,created
0,[removed],1663536087,iozbxkr,t3_xhsag5,1,1.663550e+09
1,Bro it seems you may actually delusional no ju...,1663536013,iozbqlk,t3_xhsag5,1,1.663550e+09
2,Thank you for for sharing. A reminder: if you ...,1663535972,iozbmub,t3_xhsf3f,1,1.663550e+09
3,Thanks,1663535958,iozbljf,t3_xhkzmw,1,1.663550e+09
4,I definitely have body image issues and I’m th...,1663535919,iozbi1l,t3_xhkzmw,1,1.663550e+09
...,...,...,...,...,...,...
195872,This further stigmatizes mental illness. It do...,1388387508,ced74e0,t3_1r79dm,3,1.388402e+09
195873,I hear ya. Sister suffers from bi-polar and mi...,1388367327,cecz8pw,t3_1qbx9v,2,1.388382e+09
195874,If i have to read another overwrought analogy ...,1387557036,ce6e980,t3_1r79dm,2,1.387571e+09
195875,"Ugh, that's atrocious. Just because I can't w...",1385376904,cdmivce,t3_1r79dm,7,1.385391e+09


In [238]:
all_com['body'] = all_com['body'].astype('string')

In [239]:
# removing all duplicate comments which mostly include automatic replies from bots or [removed] and [deleted] comments
all_com = all_com[~all_com.body.duplicated(keep=False)].reset_index(drop=True)

In [241]:
# list of indexes of all comments that have only one word.
one_word = list(all_com[[len(i.split())==1 for i in all_com['body'].values]].index)
# dropping all the comments that have one word only
all_com = all_com.drop(index=one_word).reset_index(drop=True)

In [243]:
# grouping all comments with the same link_id (i.e. beloning to the same submission)
all_com = pd.DataFrame(all_com.groupby('link_id')['body'].sum()).reset_index(level=0)

In [244]:
# removing 't3_' from link_id to find a match in dataframe withs submissions
all_com['link_id'] = all_com.link_id.map(lambda x: x.strip("t3_"))
all_com.rename(columns={'link_id':'id'},inplace=True)

In [252]:
# merging submissions and comments on their id
all_data = pd.merge(all_sub,all_com,how='left',on=['id'])

In [255]:
all_data = all_data[all_data['body'] != '<NA>'].reset_index(drop=True)

In [257]:
all_data.rename(columns = {'body':'comment_body'},inplace=True)
all_data.head()

,id,post_body,title,created,comment_body
0,xhrlh6,i've been having these thoughts lately like\n\...,do you ever feel like not being alive anymore,2022-09-19 00:46:32,As for anchors My only anchor is I need to be ...
1,xhrajv,i've been going through a hard time lately and...,why do depressed people shave their heads,2022-09-19 00:34:15,I don’t know because I never had the urge but ...
2,xhr49p,idk why but every now and then i'll just get t...,random urge to kill people,2022-09-19 00:27:15,You are a coward that can't handle the fact th...
3,xhq4pa,i have ocd that gives me intrusive thoughts ab...,need a therapist that specializes in concernin...,2022-09-18 23:46:40,Unless there is an actual crime or abuse to re...
4,xhpsmb,"i have no friends, i don't like my family at a...",17m i think i should end my life early,2022-09-18 23:33:01,"Aye yes, I know when I get depressed I sit in ..."


# Since we're only interested in submissions that have a question, we will need to analyze all the titles of submission and check if they are questions of not

In [260]:
question_words = ["what", "why", "when", "where", "name", "is", "how", "do", "does", 
                  "which", "are", "could", "would", "should", "has", "have", "whom", 
                  "whose", "don't",'am','can',"do i", "do you", "what", "who", "is it", 
                  "why","would you", "how","is there","are there", "is it so", "is this true" ,
                  "to know", "is that true", "are we", "am i", "question is", "tell me more", 
                  "can i", "can we", "tell me", "can you explain","didn't","doesn't","haven't","isn't","aren't","can't","couldn't","wouldn't","won't","shouldn't"]

def is_question(text):
    question = text.lower()
    question = word_tokenize(question)
    if any(x in question[0] for x in question_words):
        res = True
    else:
        res = False
    return res

In [261]:
all_data['question'] = all_data.title.apply(is_question)

In [265]:
### adapted from https://github.com/kartikn27/nlp-question-detection/blob/master/method2.py
class IsQuestion():
    
    # Init constructor
    def __init__(self):
        posts = self.__get_posts()
        feature_set = self.__get_feature_set(posts)
        self.classifier = self.__perform_classification(feature_set)
        
    # Method (Private): __get_posts
    # Input: None
    # Output: Posts (Text) from NLTK's nps_chat package
    def __get_posts(self):
        return nltk.corpus.nps_chat.xml_posts()
    
    # Method (Private): __get_feature_set
    # Input: Posts from NLTK's nps_chat package
    # Processing: 1. preserve alpha numeric characters, whitespace, apostrophe
    # 2. Tokenize sentences using NLTK's word_tokenize
    # 3. Create a dictionary of list of tuples for each post where tuples index 0 is the dictionary of words occuring in the sentence and index 1 is the class as received from nps_chat package 
    # Return: List of tuples for each post
    def __get_feature_set(self, posts):
        feature_list = []
        for post in posts:
            post_text = post.text            
            features = {}
            words = nltk.word_tokenize(post_text)
            for word in words:
                features['contains({})'.format(word.lower())] = True
            feature_list.append((features, post.get('class')))
        return feature_list
    
    # Method (Private): __perform_classification
    # Input: List of tuples for each post
    # Processing: 1. Divide data into 80% training and 10% testing sets
    # 2. Use NLTK's Multinomial Naive Bayes to perform classifcation
    # 3. Print the Accuracy of the model
    # Return: Classifier object
    def __perform_classification(self, feature_set):
        training_size = int(len(feature_set) * 0.1)
        train_set, test_set = feature_set[training_size:], feature_set[:training_size]
        classifier = nltk.NaiveBayesClassifier.train(train_set)
        print('Accuracy is : ', nltk.classify.accuracy(classifier, test_set))
        return classifier
        
    # Method (private): __get_question_words_set
    # Input: None
    # Return: Set of commonly occuring words in questions
    def __get_question_words_set(self):
        question_word_list = ['what', 'where', 'when','how','why','did','do','does','have','has','am','is','are',
                              'can','could','may','would','will','should',"has", "have", "whom",'whose',"didn't",
                              "doesn't","haven't","isn't","aren't","can't","couldn't","wouldn't","won't","shouldn't",'?',
                             "what", "why", "when", "where", "name", "is", "how", "do", "does", 
                  "which", "are", "could", "would", "should", "has", "have", "whom", 
                  "whose", "don't",'am','can',"do i", "do you", "what", "who", "is it", 
                  "why","would you", "how","is there","are there", "is it so", "is this true" ,
                  "to know", "is that true", "are we", "am i", "question is", "tell me more", 
                  "can i", "can we", "tell me", "can you explain","didn't","doesn't","haven't","isn't","aren't","can't","couldn't","wouldn't","won't","shouldn't"]
        return set(question_word_list)        
    
    # Method (Public): predict_question
    # Input: Sentence to be predicted
    # Return: 1 - If sentence is question | 0 - If sentence is not question
    def predict_question(self, text):
        words = nltk.word_tokenize(text.lower())        
        if self.__get_question_words_set().intersection(words) == False:
            return 0
        if '?' in text:
            return 1
        
        features = {}
        for word in words:
            features['contains({})'.format(word.lower())] = True            
        
        prediction_result = self.classifier.classify(features)
        if prediction_result == 'whQuestion' or prediction_result == 'ynQuestion':
            return 1
        return 0
    
    # Method (Public): predict_question_type
    # Input: Sentence to be predicted
    # Return: 'WH' - If question is WH question | 'YN' - If sentence is Yes/NO question | 'unknown' - If unknown question type
    def predict_question_type(self, text):
        words = nltk.word_tokenize(text.lower())                
        features = {}
        for word in words:
            features['contains({})'.format(word.lower())] = True            
        
        prediction_result = self.classifier.classify(features)
        if prediction_result == 'whQuestion':
            return 'WH'
        elif prediction_result == 'ynQuestion':
            return 'YN'
        else:
            return 'unknown'


isQ = IsQuestion()

Accuracy is :  0.6685606060606061


In [266]:
all_data['is_question'] = all_data['title'].apply(isQ.predict_question)

In [267]:
all_data

,id,post_body,title,created,comment_body,question,is_question
0,xhrlh6,i've been having these thoughts lately like\n\...,do you ever feel like not being alive anymore,2022-09-19 00:46:32,As for anchors My only anchor is I need to be ...,True,0
1,xhrajv,i've been going through a hard time lately and...,why do depressed people shave their heads,2022-09-19 00:34:15,I don’t know because I never had the urge but ...,True,0
2,xhr49p,idk why but every now and then i'll just get t...,random urge to kill people,2022-09-19 00:27:15,You are a coward that can't handle the fact th...,True,0
3,xhq4pa,i have ocd that gives me intrusive thoughts ab...,need a therapist that specializes in concernin...,2022-09-18 23:46:40,Unless there is an actual crime or abuse to re...,False,0
4,xhpsmb,"i have no friends, i don't like my family at a...",17m i think i should end my life early,2022-09-18 23:33:01,"Aye yes, I know when I get depressed I sit in ...",False,0
...,...,...,...,...,...,...,...
202303,2glqvl,i've been struggling with mental illness (depr...,"does mental illness, with a suicide attempt, q...",2014-09-17 02:26:12,Yes. It is a life threatening illness. You wil...,True,1
202304,2ea7v6,"so, i am getting really bad. i hear strange to...",i am scared. everything in here and on /r/depr...,2014-08-22 19:17:29,I see mental illness as a disease that gets yo...,False,0
202305,27k65x,my husband and i are both pretty fucked up rig...,losing it,2014-06-07 21:27:59,It's unfortunate to hear that one good thing l...,False,0
202306,200k7v,nan,love suicide,2014-03-10 06:24:27,Just posted a link to a blog post I did on men...,False,0


In [268]:
# since both ways i used are not perfectly accurate, I'm going to keep rows that have 'question' != False and "is_question" != 0
all_data = all_data.loc[~((all_data['question'] == False) & (all_data['is_question'] == 0))]

In [270]:
all_data = all_data.drop(columns=['question','is_question']).reset_index(drop=True)

In [271]:
all_data

,id,post_body,title,created,comment_body
0,xhrlh6,i've been having these thoughts lately like\n\...,do you ever feel like not being alive anymore,2022-09-19 00:46:32,As for anchors My only anchor is I need to be ...
1,xhrajv,i've been going through a hard time lately and...,why do depressed people shave their heads,2022-09-19 00:34:15,I don’t know because I never had the urge but ...
2,xhr49p,idk why but every now and then i'll just get t...,random urge to kill people,2022-09-19 00:27:15,You are a coward that can't handle the fact th...
3,xhowxf,"when i get really anxious, i slap my arm and t...",is slapping my arm when i’m anxious normal?,2022-09-18 22:58:06,"'Normal' and 'crazy' aren't binary opposites, ..."
4,xhovdn,i’ve been to a few psychiatrists/therapists ov...,tips to find a psychiatrist that specializes i...,2022-09-18 22:56:17,I would assume the majority are aware of anxie...
...,...,...,...,...,...
82561,2wigq2,"i'm not looking for a diagnosis, just some ins...",have you heard of this?,2015-02-20 08:05:16,I assume you have some sort of existing medica...
82562,2vr17i,how do i find friends to have adventures with ...,how do i stop my intensity?,2015-02-13 13:12:20,I have been diagnosed officially and I am not ...
82563,2s5g3q,oftain if someone will repeat a word directly ...,does this have a name?,2015-01-12 13:36:53,"I asked my councillor but he wasn't too sure, ..."
82564,2n3gpd,the patterns of loosely related random words a...,do schizophrenics 'understand' other schizophr...,2014-11-22 22:08:51,I don't think it works that way. I do find tha...


In [272]:
all_data

,id,post_body,title,created,comment_body
0,xhrlh6,i've been having these thoughts lately like\n\...,do you ever feel like not being alive anymore,2022-09-19 00:46:32,As for anchors My only anchor is I need to be ...
1,xhrajv,i've been going through a hard time lately and...,why do depressed people shave their heads,2022-09-19 00:34:15,I don’t know because I never had the urge but ...
2,xhr49p,idk why but every now and then i'll just get t...,random urge to kill people,2022-09-19 00:27:15,You are a coward that can't handle the fact th...
3,xhowxf,"when i get really anxious, i slap my arm and t...",is slapping my arm when i’m anxious normal?,2022-09-18 22:58:06,"'Normal' and 'crazy' aren't binary opposites, ..."
4,xhovdn,i’ve been to a few psychiatrists/therapists ov...,tips to find a psychiatrist that specializes i...,2022-09-18 22:56:17,I would assume the majority are aware of anxie...
...,...,...,...,...,...
82561,2wigq2,"i'm not looking for a diagnosis, just some ins...",have you heard of this?,2015-02-20 08:05:16,I assume you have some sort of existing medica...
82562,2vr17i,how do i find friends to have adventures with ...,how do i stop my intensity?,2015-02-13 13:12:20,I have been diagnosed officially and I am not ...
82563,2s5g3q,oftain if someone will repeat a word directly ...,does this have a name?,2015-01-12 13:36:53,"I asked my councillor but he wasn't too sure, ..."
82564,2n3gpd,the patterns of loosely related random words a...,do schizophrenics 'understand' other schizophr...,2014-11-22 22:08:51,I don't think it works that way. I do find tha...


In [273]:
# Dropping the columns that doesn't encode useful information and renaming the columns nicely for intepretability
all_data.drop(['id','created'], axis = 1, inplace = True)

In [282]:
# Replacing emojis with text
with open('Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

def convert_emojis_to_word(text):
    for emot in Emoji_Dict:
        text = re.sub(r'('+emot+')', "_".join(Emoji_Dict[emot].replace(",","").replace(":","").split()), text)
    return text
# Example
text = "I won 🥇 in 🏏"
convert_emojis_to_word(text)

'I won 1st_place_medal in cricket'

In [283]:
all_data['title'] = all_data['title'].apply(convert_emojis_to_word)

In [285]:
all_data['post_body'] = all_data['post_body'].apply(convert_emojis_to_word)

In [286]:
all_data['comment_body'] = all_data['comment_body'].apply(convert_emojis_to_word)

In [287]:
all_data.to_csv('../Data/all_data.csv',index=False)

# Load the data not to run the code above all over again

In [393]:
all_data = pd.read_csv('../Data/all_data.csv')

In [394]:
all_data

,post_body,title,comment_body
0,i've been having these thoughts lately like\n\...,do you ever feel like not being alive anymore,As for anchors My only anchor is I need to be ...
1,i've been going through a hard time lately and...,why do depressed people shave their heads,I don’t know because I never had the urge but ...
2,idk why but every now and then i'll just get t...,random urge to kill people,You are a coward that can't handle the fact th...
3,"when i get really anxious, i slap my arm and t...",is slapping my arm when i’m anxious normal?,"'Normal' and 'crazy' aren't binary opposites, ..."
4,i’ve been to a few psychiatrists/therapists ov...,tips to find a psychiatrist that specializes i...,I would assume the majority are aware of anxie...
...,...,...,...
82561,"i'm not looking for a diagnosis, just some ins...",have you heard of this?,I assume you have some sort of existing medica...
82562,how do i find friends to have adventures with ...,how do i stop my intensity?,I have been diagnosed officially and I am not ...
82563,oftain if someone will repeat a word directly ...,does this have a name?,"I asked my councillor but he wasn't too sure, ..."
82564,the patterns of loosely related random words a...,do schizophrenics 'understand' other schizophr...,I don't think it works that way. I do find tha...


In [395]:
# lower all text
all_data = all_data.apply(lambda x: x.str.lower())

In [396]:
# convert to string
all_data = all_data.astype('string')

In [397]:
# replace all duplicated post_body with null values
duplicated = all_data.duplicated('post_body')
all_data.loc[duplicated, ['post_body']] = " "

In [398]:
# merge title and post_body
all_data['title'] = all_data['title'] +' '  + all_data['post_body']

In [399]:
all_data.drop(columns=['post_body'],inplace=True)

In [411]:
all_data = all_data.dropna()

In [413]:
# remove punctuation
import string
punctuation = ['\u200d','?', '....','..','...','','@','#', ',', '.', '"', ':', ')', '(', '-', '!', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '*', '+', '\\', 
    '•', '~', '£', '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', 
    '½', 'à', '…', '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', 
    '—', '‹', '─', '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 
    'Ã', '⋅', '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', 
    '¹', '≤', '‡', '√', '!','🅰','🅱']
def remove_punct(string):
    for el in string:
        if el in punctuation:
            string = string.replace(el, "")
    return string
all_data['title'] = all_data.title.apply(lambda x: remove_punct(x))
all_data['comment_body'] = all_data.comment_body.apply(lambda x: remove_punct(x))

C:\Users\Julia\AppData\Local\Temp\ipykernel_14500\1271528730.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data['title'] = all_data.title.apply(lambda x: remove_punct(x))
C:\Users\Julia\AppData\Local\Temp\ipykernel_14500\1271528730.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data['comment_body'] = all_data.comment_body.apply(lambda x: remove_punct(x))


In [415]:
# remove digits
all_data['title'] = all_data['title'].str.replace('\d+', '')
all_data['comment_body'] = all_data['comment_body'].str.replace('\d+', '')

<>:2: DeprecationWarning: invalid escape sequence \d
<>:3: DeprecationWarning: invalid escape sequence \d
<>:2: DeprecationWarning: invalid escape sequence \d
<>:3: DeprecationWarning: invalid escape sequence \d
C:\Users\Julia\AppData\Local\Temp\ipykernel_14500\712171288.py:2: DeprecationWarning: invalid escape sequence \d
  all_data['title'] = all_data['title'].str.replace('\d+', '')
C:\Users\Julia\AppData\Local\Temp\ipykernel_14500\712171288.py:3: DeprecationWarning: invalid escape sequence \d
  all_data['comment_body'] = all_data['comment_body'].str.replace('\d+', '')
C:\Users\Julia\AppData\Local\Temp\ipykernel_14500\712171288.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  all_data['title'] = all_data['title'].str.replace('\d+', '')
C:\Users\Julia\AppData\Local\Temp\ipykernel_14500\712171288.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [416]:
all_data

,title,comment_body
0,do you ever feel like not being alive anymorei...,as for anchors my only anchor is i need to be ...
1,why do depressed people shave their headsive b...,i dont know because i never had the urge but i...
2,random urge to kill peopleidk why but every no...,you are a coward that cant handle the fact tha...
3,is slapping my arm when im anxious normalwhen ...,normal and crazy arent binary opposites i woul...
4,tips to find a psychiatrist that specializes i...,i would assume the majority are aware of anxie...
...,...,...
82561,have you heard of thisim not looking for a dia...,i assume you have some sort of existing medica...
82562,how do i stop my intensityhow do i find friend...,i have been diagnosed officially and i am not ...
82563,does this have a nameoftain if someone will re...,i asked my councillor but he wasnt too sure it...
82564,do schizophrenics understand other schizophren...,i dont think it works that way i do find that ...


In [ ]:
# tokenize all text
def word_tokenize(sentence):
    return sentence.split()
all_data = all_data.apply(lambda x: word_tokenize(x.str))

In [ ]:
new

In [ ]:
new_string = a_string.translate(str.maketrans('', '', string.punctuation))

In [ ]:
# remove numbers

In [ ]:
# remove emojis that weren't converted into text

In [ ]:
# remove digits

In [ ]:
# Writing a function to plot the counts that are in the data

def top10_bagofwords(data, title):
    ''' Taking as input the data and plots the top 10 words based on counts in this text data'''
    bagofwords = CountVectorizer()
    inbound = bagofwords.fit_transform(data)
    inbound # Output will be a sparse matrix
    # Inspecting of often contractions and colloquial language is used
    word_counts = np.array(np.sum(inbound, axis=0)).reshape((-1,))
    words = np.array(bagofwords.get_feature_names())
    words_df = pd.DataFrame({"word":words, 
                             "count":word_counts})
    words_rank = words_df.sort_values(by="count", ascending=False)
    # words_rank.to_csv('words_rank.csv') # Storing it in a csv so I can inspect and go through it myself
    words_rank.head()
    # Visualizing top 10 words
    #plt.figure(figsize=(12,6))
    #sns.barplot(words_rank['word'][:10], words_rank['count'][:10].astype(str), palette = 'inferno')
    #plt.title(title) 
    #plt.show()
    return words_rank

In [ ]:
top10_bagofwords(all_data.title, "top")

# Processing emojis
# Since emojis can give information about a text

In [ ]:
all_sub['title'] = all_sub['title'].apply(convert_emojis_to_word)

In [ ]:
all_sub['post_body'] = all_sub['post_body'].apply(convert_emojis_to_word)

In [ ]:
demoji.download_codes()

In [ ]:
def remove_em(text):
    dem = demoji.findall(text)
    for item in dem.keys():
        text = text.replace(item, "")
    return text

In [ ]:
all_sub['title'] = all_sub['title'].apply(remove_em)

In [ ]:
all_sub['post_body'] = all_sub['post_body'].apply(remove_em)